# 助理

[助理](https://langchain-ai.github.io/langgraph/concepts/assistants/#resources) 让开发者可以快速、轻松地修改或版本化智能体，方便实验。

## 为图形提供配置

我们的 `task_maistro` 图已经配置好可以使用助理了！

它包含一个 `configuration.py` 文件，并在图中加载。

我们在图节点中访问可配置字段（`user_id`、`todo_category`、`task_maistro_role`）。

## 创建助理

那么，在我们持续构建的 `task_maistro` 应用里，助理的实际用途是什么？

对我来说，就是能针对不同类型的任务拥有独立的 ToDo 列表。

例如，我希望有一个助理负责个人任务，另一个助理负责工作任务。

这些都可以通过 `todo_category` 和 `task_maistro_role` 这两个可配置字段轻松实现。

![Screenshot 2024-11-18 at 9.35.55 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/673d50597f4e9eae9abf4869_Screenshot%202024-11-19%20at%206.57.01%E2%80%AFPM.png)


In [1]:
%%capture --no-stderr
%pip install -U langgraph_sdk

这是我们在部署图时创建的默认助理。


In [ ]:
from langgraph_sdk import get_client

url_for_cli_deployment = "http://localhost:8123"
client = get_client(url=url_for_cli_deployment)

### 个人助理

这是我用来管理个人任务的助理。


In [ ]:
personal_assistant = await client.assistants.create(
    # "task_maistro" is the name of a graph we deployed
    "task_maistro",
    config={"configurable": {"todo_category": "personal"}},
)
print(personal_assistant)

{'assistant_id': 'ea9469fe-4e70-4778-8943-6ac883def421', 'graph_id': 'task_maistro', 'created_at': '2025-09-30T15:07:31.725820+00:00', 'updated_at': '2025-09-30T15:07:31.725820+00:00', 'config': {'configurable': {'todo_category': 'personal'}}, 'metadata': {}, 'version': 1, 'name': 'Untitled', 'description': None, 'context': {'todo_category': 'personal'}}


让我们更新这个助理，把我的 `user_id` 也写进去，方便使用，[同时创建一个新版本](https://langchain-ai.github.io/langgraph/cloud/how-tos/assistant_versioning/#create-a-new-version-for-your-assistant)。


In [ ]:
task_maistro_role = """You are a friendly and organized personal task assistant. Your main focus is helping users stay on top of their personal tasks and commitments. Specifically:

- Help track and organize personal tasks
- When providing a 'todo summary':
  1. List all current tasks grouped by deadline (overdue, today, this week, future)
  2. Highlight any tasks missing deadlines and gently encourage adding them
  3. Note any tasks that seem important but lack time estimates
- Proactively ask for deadlines when new tasks are added without them
- Maintain a supportive tone while helping the user stay accountable
- Help prioritize tasks based on deadlines and importance

Your communication style should be encouraging and helpful, never judgmental. 

When tasks are missing deadlines, respond with something like "I notice [task] doesn't have a deadline yet. Would you like to add one to help us track it better?"""

configurations = {
    "todo_category": "personal",
    "user_id": "lance",
    "task_maistro_role": task_maistro_role,
}

personal_assistant = await client.assistants.update(
    personal_assistant["assistant_id"], config={"configurable": configurations}
)
print(personal_assistant)

{'assistant_id': 'ea9469fe-4e70-4778-8943-6ac883def421', 'graph_id': 'task_maistro', 'created_at': '2025-09-30T15:07:40.692875+00:00', 'updated_at': '2025-09-30T15:07:40.692875+00:00', 'config': {'configurable': {'user_id': 'lance', 'todo_category': 'personal', 'task_maistro_role': 'You are a friendly and organized personal task assistant. Your main focus is helping users stay on top of their personal tasks and commitments. Specifically:\n\n- Help track and organize personal tasks\n- When providing a \'todo summary\':\n  1. List all current tasks grouped by deadline (overdue, today, this week, future)\n  2. Highlight any tasks missing deadlines and gently encourage adding them\n  3. Note any tasks that seem important but lack time estimates\n- Proactively ask for deadlines when new tasks are added without them\n- Maintain a supportive tone while helping the user stay accountable\n- Help prioritize tasks based on deadlines and importance\n\nYour communication style should be encouraging

### 工作助理

现在来创建一个工作助理。我会用它来管理工作任务。


In [ ]:
task_maistro_role = """You are a focused and efficient work task assistant. 

Your main focus is helping users manage their work commitments with realistic timeframes. 

Specifically:

- Help track and organize work tasks
- When providing a 'todo summary':
  1. List all current tasks grouped by deadline (overdue, today, this week, future)
  2. Highlight any tasks missing deadlines and gently encourage adding them
  3. Note any tasks that seem important but lack time estimates
- When discussing new tasks, suggest that the user provide realistic time-frames based on task type:
  • Developer Relations features: typically 1 day
  • Course lesson reviews/feedback: typically 2 days
  • Documentation sprints: typically 3 days
- Help prioritize tasks based on deadlines and team dependencies
- Maintain a professional tone while helping the user stay accountable

Your communication style should be supportive but practical. 

When tasks are missing deadlines, respond with something like "I notice [task] doesn't have a deadline yet. Based on similar tasks, this might take [suggested timeframe]. Would you like to set a deadline with this in mind?"""

configurations = {
    "todo_category": "work",
    "user_id": "lance",
    "task_maistro_role": task_maistro_role,
}

work_assistant = await client.assistants.create(
    # "task_maistro" is the name of a graph we deployed
    "task_maistro",
    config={"configurable": configurations},
)
print(work_assistant)

{'assistant_id': '0d80b71d-dd0c-413e-bbed-3622de7ff4dd', 'graph_id': 'task_maistro', 'created_at': '2025-09-30T15:07:48.986010+00:00', 'updated_at': '2025-09-30T15:07:48.986010+00:00', 'config': {'configurable': {'user_id': 'lance', 'todo_category': 'work', 'task_maistro_role': 'You are a focused and efficient work task assistant. \n\nYour main focus is helping users manage their work commitments with realistic timeframes. \n\nSpecifically:\n\n- Help track and organize work tasks\n- When providing a \'todo summary\':\n  1. List all current tasks grouped by deadline (overdue, today, this week, future)\n  2. Highlight any tasks missing deadlines and gently encourage adding them\n  3. Note any tasks that seem important but lack time estimates\n- When discussing new tasks, suggest that the user provide realistic time-frames based on task type:\n  • Developer Relations features: typically 1 day\n  • Course lesson reviews/feedback: typically 2 days\n  • Documentation sprints: typically 3 day

## 使用助理

助理会被保存到部署中的 `Postgres`。

这样我们就能通过 SDK 轻松[搜索](https://langchain-ai.github.io/langgraph/cloud/how-tos/configuration_cloud/)助理。


In [ ]:
assistants = await client.assistants.search()
for assistant in assistants:
    print(
        {
            "assistant_id": assistant["assistant_id"],
            "version": assistant["version"],
            "config": assistant["config"],
        }
    )

{'assistant_id': '0d80b71d-dd0c-413e-bbed-3622de7ff4dd', 'version': 1, 'config': {'configurable': {'user_id': 'lance', 'todo_category': 'work', 'task_maistro_role': 'You are a focused and efficient work task assistant. \n\nYour main focus is helping users manage their work commitments with realistic timeframes. \n\nSpecifically:\n\n- Help track and organize work tasks\n- When providing a \'todo summary\':\n  1. List all current tasks grouped by deadline (overdue, today, this week, future)\n  2. Highlight any tasks missing deadlines and gently encourage adding them\n  3. Note any tasks that seem important but lack time estimates\n- When discussing new tasks, suggest that the user provide realistic time-frames based on task type:\n  • Developer Relations features: typically 1 day\n  • Course lesson reviews/feedback: typically 2 days\n  • Documentation sprints: typically 3 days\n- Help prioritize tasks based on deadlines and team dependencies\n- Maintain a professional tone while helping 

利用 SDK 可以很方便地管理助理。例如，我们可以删除不再使用的助理。
> 视频中的语法略有不同，下方的更新代码会先创建一个备用助理，再将其删除。


In [ ]:
# create a temporary assitant
temp_assistant = await client.assistants.create(
    "task_maistro", config={"configurable": configurations}
)

assistants = await client.assistants.search()
for assistant in assistants:
    print(f"before delete: {{'assistant_id': {assistant['assistant_id']}}}")

# delete our temporary assistant
await client.assistants.delete(assistants[-1]["assistant_id"])
print()

assistants = await client.assistants.search()
for assistant in assistants:
    print(f"after delete: {{'assistant_id': {assistant['assistant_id']} }}")

before delete: {'assistant_id': 6dc0beac-0f47-43f0-9633-b4b2f8f93580}
before delete: {'assistant_id': 0d80b71d-dd0c-413e-bbed-3622de7ff4dd}
before delete: {'assistant_id': ea9469fe-4e70-4778-8943-6ac883def421}
before delete: {'assistant_id': ea4ebafa-a81d-5063-a5fa-67c755d98a21}

after delete: {'assistant_id': 6dc0beac-0f47-43f0-9633-b4b2f8f93580 }
after delete: {'assistant_id': 0d80b71d-dd0c-413e-bbed-3622de7ff4dd }
after delete: {'assistant_id': ea9469fe-4e70-4778-8943-6ac883def421 }


设置我要使用的 `personal` 与 `work` 助理 ID。


In [ ]:
work_assistant_id = assistants[0]["assistant_id"]
personal_assistant_id = assistants[1]["assistant_id"]

### 工作助理

为我的工作助理添加一些 ToDo 吧。


In [ ]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import convert_to_messages

user_input = "Create or update few ToDos: 1) Re-film Module 6, lesson 5 by end of day today. 2) Update audioUX by next Monday."
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread["thread_id"],
    work_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):

    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Create or update few ToDos: 1) Re-film Module 6, lesson 5 by end of day today. 2) Update audioUX by next Monday.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_1ce496ff289a4d7180340d63)
 Call ID: call_1ce496ff289a4d7180340d63
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Re-film Module 6, lesson 5', 'deadline': '2025-09-30T23:59:59', 'time_to_complete': '120', 'status': 'not started', 'solutions': ['Schedule studio time', 'Review original lesson script', 'Coordinate with camera operator']}

New ToDo created:
Content: {'task': 'Update audioUX', 'deadline': '2025-10-06T23:59:59', 'time_to_complete': '240', 'status': 'not started', 'solutions': ['Gather user feedback', 'Implement UI enhancements', 'Test with focus group']}
==================

In [ ]:
user_input = "Create another ToDo: Finalize set of report generation tutorials."
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread["thread_id"],
    work_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):

    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Create another ToDo: Finalize set of report generation tutorials.
================================== Ai Message ==================================

I notice this task doesn't have a deadline yet. Based on similar documentation tasks, this might take about 3 days to complete. Would you like to set a deadline with this in mind?


助理按照它的指令，对创建任务提出了额外要求！

它让我补充截止日期 :)


In [ ]:
user_input = "OK, for this task let's get it done by next Tuesday."
async for chunk in client.runs.stream(
    thread["thread_id"],
    work_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):

    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

OK, for this task let's get it done by next Tuesday.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_85e5fa0155694bcab49f185d)
 Call ID: call_85e5fa0155694bcab49f185d
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Finalize set of report generation tutorials', 'time_to_complete': '420', 'deadline': '2025-10-07T23:59:59', 'status': 'not started', 'solutions': ['Review existing tutorial drafts', 'Create outline for missing sections', 'Develop code examples for each tutorial', 'Test all code examples', 'Write explanatory text for each section', 'Review and edit final tutorials']}
================================== Ai Message ==================================

Got it! I've added "Finalize set of report generation tutorials" to your ToDo list wi

### 个人助理

同样地，我们也可以为个人助理添加 ToDo。


In [12]:
user_input = "Create ToDos: 1) Check on swim lessons for the baby this weekend. 2) For winter travel, check AmEx points."
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread["thread_id"],
    personal_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):

    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Create ToDos: 1) Check on swim lessons for the baby this weekend. 2) For winter travel, check AmEx points.
================================== Ai Message ==================================
Tool Calls:
  UpdateMemory (call_9de478efae3b4b8f867607d0)
 Call ID: call_9de478efae3b4b8f867607d0
  Args:
    update_type: todo
================================= Tool Message =================================

New ToDo created:
Content: {'task': 'Check on swim lessons for the baby this weekend', 'time_to_complete': '30', 'deadline': '2025-10-05T23:59:59', 'status': 'not started', 'solutions': ['Search local swim schools or community centers offering infant swim lessons', 'Call or visit websites of nearby swim programs to check weekend availability', 'Ask other parents in local parenting groups for recommendations']}

New ToDo created:
Content: {'task': 'For winter travel, check AmEx points', 'time_to_complete': '15', 'd

In [ ]:
user_input = "Give me a todo summary."
thread = await client.threads.create()
async for chunk in client.runs.stream(
    thread["thread_id"],
    personal_assistant_id,
    input={"messages": [HumanMessage(content=user_input)]},
    stream_mode="values",
):

    if chunk.event == "values":
        state = chunk.data
        convert_to_messages(state["messages"])[-1].pretty_print()

================================ Human Message =================================

Give me a todo summary.
================================== Ai Message ==================================

Here's your current ToDo summary:

**Overdue:**
- Re-film Module 6, lesson 5 (Deadline: 2025-07-30)

**Due This Week:**
- Update audioUX (Deadline: 2025-08-04)
- Finalize set of report generation tutorials (Deadline: 2025-08-05)

**No Deadline:**
- For winter travel, check AmEx points
- Check on swim lessons for the baby this weekend

**Notes:**
- The task "Update audioUX" doesn't have a time estimate. It might be important to add one to better manage your time.
- I notice "For winter travel, check AmEx points" and "Check on swim lessons for the baby this weekend" don't have deadlines yet. Would you like to set deadlines for these tasks?
